In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import stanza
stanza.download('en') # download English model
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

2021-03-07 18:32:14 INFO: Downloading default packages for language: en (English)...
2021-03-07 18:32:15 INFO: File exists: C:\Users\vibkr\stanza_resources\en\default.zip.
2021-03-07 18:32:18 INFO: Finished downloading models and saved to C:\Users\vibkr\stanza_resources.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vibkr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vibkr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vibkr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def feature_extraction(txt, nlp):
    try:
        txt = txt.lower()
    except:
        pass;

    sentList = nltk.sent_tokenize(txt)

    retlist = [];
    
    for line in sentList:
        txt_list = nltk.word_tokenize(line)
        taggedList = nltk.pos_tag(txt_list)
        
        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])
        finaltxt = ' '.join(word for word in newwordList)
    
    
        stop_words = set(stopwords.words('english'))
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(wordsList)
        
        doc = nlp(finaltxt)
        dep_node = []
        try:
            for dep_edge in doc.sentences[0].dependencies:
                dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])
            for i in range(0, len(dep_node)):
                if (int(dep_node[i][1]) != 0):
                    dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]
        except:
            pass;
        
        #print(dep_node)
        
        featureList = []
        categories = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i))
                categories.append(i[0])
        #print(featureList)
        #print(categories)
        
        
        fcluster = []
        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
        print(fcluster) 
        
        # Remove all features with no sentiment word:
        
        retlist.append(fcluster)
    return retlist;
    

In [3]:
def do_extraction(df, nlp, content_str = "Content"):
    idx = 0;
    review_list = df[content_str].to_list()
    feat_count = dict()
    feat_sent = dict()
    #nlp = stanza.Pipeline('en')
    
    # Replace "" with nan's for removal
    df[content_str].replace('', np.nan, inplace=True)
    df.dropna(subset=[content_str], inplace=True)
    print(" Processing : " , df.shape[0], "rows of data")
    for review in review_list:
        print("Review Number : ", idx);
        idx += 1;
        if idx >= df.shape[0]:
            break;
        try:
            output = feature_extraction(review, nlp);
        except:
            pass;
        for sent in output:
            for pair in sent:
                print(pair)
                if pair[0] in feat_sent:
                    if pair[1] is not None:
                        flist = feat_sent[pair[0]]
                        if isinstance(pair[1], list):
                            for i in pair[1]:
                                flist.append(i)
                        else:
                            flist.append(pair[1])
                        feat_sent[pair[0]] = flist;
                else:
                    if pair[1] is not None:
                        flist = pair[1]
                    else:
                        flist = list()
                    feat_sent[pair[0]] = flist;
                
                if pair[0] in feat_count:
                    feat_count[pair[0]] = feat_count[pair[0]] + 1;
                else:
                    feat_count[pair[0]] = 1

    #print(feat_count);
    return feat_count, feat_sent;

In [4]:
def get_sentiment(a, b, nlp):

    sentiment_score = dict()

    # Delete features with no descriptors
    cob = b.copy()
    for feat in cob.keys():
        #print(cob[feat])
        if cob[feat] == []:
            del b[feat]

    # Run pre-built sentiment score and take avg of all descriptors
    for f in b.keys():
        print(f);
        ssum = 0;
        for g in b[f]:
            try:
                doc = nlp(g);

                for i in doc.sentences:

                        #print(i.sentiment)
                        ssum += i.sentiment;
            except:
                pass;

        sentiment_score[f] = ssum / len(b[f])

    adf = pd.DataFrame.from_dict(a, orient='index', columns=['Freq'])
    adf.sort_values(by="Freq", ascending=False, inplace = True)



    avg_sent = pd.DataFrame.from_dict(sentiment_score, orient='index', columns=["Avg_sent"])

    final_sent = avg_sent.merge(adf, left_index=True, right_index=True)
    final_sent.sort_values(by="Freq", ascending=False, inplace=True)
    return final_sent;

In [5]:
rdr = pd.read_csv('../ScrapedOutput/cmpb.csv')

nlp = stanza.Pipeline('en')

a, b = do_extraction(rdr, nlp)
final_sent = get_sentiment(a, b, nlp)

2021-03-07 18:32:19 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-03-07 18:32:19 INFO: Use device: cpu
2021-03-07 18:32:19 INFO: Loading: tokenize
2021-03-07 18:32:19 INFO: Loading: pos
2021-03-07 18:32:19 INFO: Loading: lemma
2021-03-07 18:32:19 INFO: Loading: depparse
2021-03-07 18:32:19 INFO: Loading: sentiment
2021-03-07 18:32:19 INFO: Loading: ner
2021-03-07 18:32:20 INFO: Done loading processors!


 Processing :  52 rows of data
Review Number :  0
[['professional', ['very', 'people']], ['people', ['professional', 'there']]]
[['recommended⭐⭐⭐⭐⭐', ['kind']], ['patient', ['very', 'kind']], ['kind', ['recommended⭐⭐⭐⭐⭐', 'patient', 'respectful']], ['respectful', ['kind']], ['..', []], ['smooth', ['very']], ['medical', [14]], ['check-up', []]]
['professional', ['very', 'people']]
['people', ['professional', 'there']]
['recommended⭐⭐⭐⭐⭐', ['kind']]
['patient', ['very', 'kind']]
['kind', ['recommended⭐⭐⭐⭐⭐', 'patient', 'respectful']]
['respectful', ['kind']]
['..', []]
['smooth', ['very']]
['medical', [14]]
['check-up', []]
Review Number :  1
[['reviews', ['other', 'suggest']], ['people', ['friendly', 'here']], ['adequately', ['friendly']], ['friendly', ['people', 'adequately']], ['anywhere', ['else']], ['else', ['anywhere']], ['sg', []]]
[['place', ['clean']], ['really', ['clean']], ['clean', ['place', 'really']], ['efficient', []]]
['reviews', ['other', 'suggest']]
['people', ['friendl

[['probably', ['this']], ['…', []]]
['ok', []]
['lah', []]
['review', []]
['visit', []]
['january', []]
['maybe', ['sikit']]
['sikit', ['so', 'maybe']]
['date', []]
['nsf', ['staff']]
['staff', ['nsf', 'ok']]
['ok', ['staff']]
['typical', ['bochap']]
['bochap', ['typical']]
['happy', []]
['bird', []]
['tio', ['switch', 'vocation']]
['switch', ['tio']]
['vocation', ['tio']]
['mo', ['seemed']]
['hand', ['other']]
['si', []]
['pehbuay', ['buaysong']]
['buaysong', ['pehbuay', 'seemed']]
['probably', ['this']]
['…', []]
Review Number :  14
[['tbh', []], ['bad', ['not', 'that']], ['place', []], ['visit', []], ['contrary', []], ['others', ['saying']]]
['tbh', []]
['bad', ['not', 'that']]
['place', []]
['visit', []]
['contrary', []]
['others', ['saying']]
Review Number :  15
[['enlisting', ['here']], ['sure', ['make']], ['medical', ['conditions']], ['conditions', ['medical', 'declare']]]
[['severe', []], ['minor', []], ['medical', ['officer']], ['officer', ['medical']], ['checkup', []]]
[['bel

[['cookhouse', []], ['book', ['get', 'everyday']], ['everyday', ['book']]]
[['troublesome', ['most', 'thing']], ['thing', ['troublesome', 'discussing']], ['discussing', ['thing', 'what']], ['lunch', []]]
[['sidenote', []], ['菜贩', ['canteenb']], ['@', []], ['canteen', []], ['canteenb', ['菜贩', '@canteen', 'bad']], ['bad', ['canteenb', 'real']], ['real', ['bad']], ['bad', ['canteenb', 'real']]]
['cookhouse', []]
['book', ['get', 'everyday']]
['everyday', ['book']]
['troublesome', ['most', 'thing']]
['thing', ['troublesome', 'discussing']]
['discussing', ['thing', 'what']]
['lunch', []]
['sidenote', []]
['菜贩', ['canteenb']]
['@', []]
['canteen', []]
['canteenb', ['菜贩', '@canteen', 'bad']]
['bad', ['canteenb', 'real']]
['real', ['bad']]
['bad', ['canteenb', 'real']]
Review Number :  38
[['people', ['go']], ['even', ['go']], ['place', []], ['middle', []], ['nowhere', []]]
['people', ['go']]
['even', ['go']]
['place', []]
['middle', []]
['nowhere', []]
Review Number :  39
[['officertalk', ['p

In [6]:
final_sent

,Avg_sent,Freq
staff,0.909091,11
medical,1.200000,10
people,1.250000,10
bad,1.058824,8
n't,1.300000,8
...,...,...
directions,0.000000,1
expect,1.000000,1
whole,1.000000,1
process,1.000000,1


In [7]:
fdr = pd.read_csv('../ScrapedOutput/nationalservicesg_comments.csv')
nlp = stanza.Pipeline('en')

c, d = do_extraction(fdr[0:20], nlp, "body")
final_sent2 = get_sentiment(c, d, nlp)

2021-03-07 18:33:00 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-03-07 18:33:00 INFO: Use device: cpu
2021-03-07 18:33:00 INFO: Loading: tokenize
2021-03-07 18:33:00 INFO: Loading: pos
2021-03-07 18:33:00 INFO: Loading: lemma
2021-03-07 18:33:00 INFO: Loading: depparse
2021-03-07 18:33:01 INFO: Loading: sentiment
2021-03-07 18:33:01 INFO: Loading: ner
2021-03-07 18:33:01 INFO: Done loading processors!


 Processing :  804 rows of data
Review Number :  0
[['mo', []]]
[['ever', ['have']], ['haunt', ['you']], ['age', []]]
['mo', []]
['ever', ['have']]
['haunt', ['you']]
['age', []]
Review Number :  1
[['legit', ['means']], ['rsi', ['ba']], ['ba', ['rsi']]]
[['u', ['want']], ['wait', ['want']], ['bookout', ['you']], ['also', ['diff']], ['diff', ['also']], ['superiors', []]]
[['rsi', ['just']]]
['legit', ['means']]
['rsi', ['ba']]
['ba', ['rsi']]
['u', ['want']]
['wait', ['want']]
['bookout', ['you']]
['also', ['diff']]
['diff', ['also']]
['superiors', []]
['rsi', ['just']]
Review Number :  2
[['mean', ['i']], ['really', ['worried']], ['bad', ['impression']], ['impression', ['bad']], ['weekend', []], ['let', ['them', 'know']], ['’', []], ['bookout', ['you']], ['keng', ['time']], ['time', ['keng', 'trying']]]
[['mean', ['i']], ['health', ['neglect']], ['impression', []], ['’', []], ['superiors', []]]
['mean', ['i']]
['really', ['worried']]
['bad', ['impression']]
['impression', ['bad']]
['w

[['’', []], ['even', ['possible']], ['possible', ['even']]]
['’', []]
['even', ['possible']]
['possible', ['even']]
Review Number :  20
[['draw', ['had', 'ic']], ['ic', ['draw', 'new']], ['new', ['ic']], ['ic', ['draw', 'new']], ['change', []], ['particulars', []]]
[['icdepartment/cmpb', []], ['enquire', []], ['ur', []], ['ic', ['ic']], ['valid', ['relevant', 'reason']], ['reason', ['valid']], ['relevant', ['valid', 'do']], ['document', []]]
['draw', ['had', 'ic']]
['ic', ['draw', 'new']]
['new', ['ic']]
['ic', ['draw', 'new']]
['change', []]
['particulars', []]
['icdepartment/cmpb', []]
['enquire', []]
['ur', []]
['ic', ['ic']]
['valid', ['relevant', 'reason']]
['reason', ['valid']]
['relevant', ['valid', 'do']]
['document', []]
Review Number :  21
[['long', ['not', 'enough']], ['enough', ['long']], ['complain', []]]
[['guy', ['encountered']], ['didnt', []], ['complain', ['he']], ['abt', []], ['collectiontime', []], ['id', ['assume']], ['<', []], ['=15mins', []]]
['long', ['not', 'eno

[['engineers', []], ['first', ['weeks']], ['weeks', ['first']], ['nee', []], ['soon', ['camp']], ['camp', ['soon']], ['counter', ['training']], ['training', ['counter', 'ied', 'undergoing']]]
[['badge', ['cied', 'get']], ['weeks', []]]
[['learn', ['you']], ['ignite', []], ['live', ['charges']], ['charges', ['live', 'prepare']], ['time', []]]
[['outfields', []], ['packsense', ['fieldpack']], ['outfield', ['training']], ['training', ['outfield', 'is']]]
[['average', ['day']], ['day', ['average']], ['time', []], ['last', ['year']], ['year', ['last']]]
[['high', ['keys']], ['keys', ['high', 'include']], ['demo', ['firing']], ['live', ['firing']], ['firing', ['demo', 'live', 'include']], ['summex', []]]
[['weeks', []], ['split', []], ['specializations', ['further']], ['armour', []], ['field', []], ['plant', []], ['bridging', []], ['cbrd', []], ['eod', []]]
[['batch', ['bridging', 'cbre']], ['bridging', ['batch']], ['cbre', ['batch']], ['batch', ['bridging', 'cbre']]]
[['next', ['batch']], [

[['personally', ['rejected']], ['command', ['going', 'school']], ['school', ['command']], ['leadership', ['system']], ['system', ['leadership', 'hated']], ['saf', []]]
[['proven', ['right']], ['right', ['proven']], ['unit', ['go']], ['men', ['seeing', 'lead']], ['better', ['lead']], ['commanders', ['it']], ['sgt', []], ['officer', []]]
['thing', ['same', 'applies']]
['applies', ['thing']]
['result', ['also']]
['also', ['result']]
['ippt', []]
['sit', ['then', 'then', 'test', 'appraisal']]
['test', ['sit']]
['peer', ['appraisal']]
['appraisal', ['then', 'peer', 'sit']]
['leadership', ['qualities']]
['qualities', ['leadership', 'impt']]
['impt', ['qualities', 'also']]
['also', ['impt']]
['form', ['just']]
['initiative', ['taking']]
['point', ['not', 'find']]
['section', ['mates']]
['annoying', ['find']]
['fieldcamp', ['leadership']]
['leadership', ['fieldcamp', 'important']]
['important', ['leadership', 'more']]
['normal', ['days']]
['days', ['normal']]
['commanders', ['keep']]
['really'

...

[['way', ['best', 'make']], ['sane/happy', []], ['ns', []], ['make', ['way']]]
[['dont', []], ['much', ['know']], ['ns', ['life']], ['life', ['ns', 'chill']], ['really', ['chill']], ['chill', ['life', 'really']], ['8-5', []], ['job', [17]]]
[['ns', ['entered']], ['ambitions', []], ['even', ['wanted']], ['wanted', ['even', 'sign']], ['sign', ['wanted']]]
[['god', ['thank']], ['didnt', ['i']], ['however', ['had']], ['circuitbreaker', ['breakerperiod']], ['breakerperiod', ['circuitbreaker']], ['chance', ['had']], ['sit', []], ['wanted', ['what', 'i', 'do']], ['life', []], ['practical', ['steps']], ['steps', ['practical']], ['direction', []]]
[['also', ['learnt']], ['learnt', ['i', 'also']], ['financially', ['more', 'literate']], ['literate', ['financially']]]
[['ask', ['you', 'me', 'sane']], ['sane', ['ask']], ['ns', []]]
[['online', ['courses']], ['courses', ['online', 'taking']], ['new', ['hobbies']], ['hobbies', ['new', 'finding']]]
[['selfdiscovery', []]]
['heyman', []]
['way', ['best

[]
[['switched', ['i', 'brain']], ['brain', ['switched']], ['crap', ['take']], ['way', ['coming']]]
[['friends', ['make']], ['everybody', []], ['name', ['learn']]]
[['alone', ['feeling']], ['feeling', ['you', 'not', 'alone', 'way']], ['way', ['feeling', 'are']]]
[['see', ['tend', 'side']], ['funny', ['side']], ['side', ['funny', 'see']], ['everything', []]]
[['get', ['screamed']], ['face', []], ['inside', ['laughing']], ['lol', []]]
[['hang', []], ['buddy', []]]
['dad', ['gave']]
['good', ['advice']]
['advice', ['good', 'gave']]
['treat', ['it']]
['holiday', []]
['switched', ['i', 'brain']]
['brain', ['switched']]
['crap', ['take']]
['way', ['coming']]
['friends', ['make']]
['everybody', []]
['name', ['learn']]
['alone', ['feeling']]
['feeling', ['you', 'not', 'alone', 'way']]
['way', ['feeling', 'are']]
['see', ['tend', 'side']]
['funny', ['side']]
['side', ['funny', 'see']]
['everything', []]
['get', ['screamed']]
['face', []]
['inside', ['laughing']]
['lol', []]
['hang', []]
['buddy

[['u', ['get']], ['almost', ['get']], ['bran', ['injury']], ['injury', ['bran', 'get']]]
['u', ['get']]
['almost', ['get']]
['bran', ['injury']]
['injury', ['bran', 'get']]
Review Number :  784
[['yes', []]]
[['saf', ['give']], ['give', ['saf', 'c', 'fk']], ['c', ['give']], ['fk', ['then', 'give']], ['alr', []]]
['yes', []]
['saf', ['give']]
['give', ['saf', 'c', 'fk']]
['c', ['give']]
['fk', ['then', 'give']]
['alr', []]
Review Number :  785
[['also', ['gotten']], ['gotten', ['i', 'also', 'braindamage']], ['braindamage', ['gotten']], ['army', []], ['everyday', ['feel']], ['feel', ['everyday', 'braincell', 'dying']], ['braincell', ['feel']], ['sure', ['not']], ['tho', []]]
['also', ['gotten']]
['gotten', ['i', 'also', 'braindamage']]
['braindamage', ['gotten']]
['army', []]
['everyday', ['feel']]
['feel', ['everyday', 'braincell', 'dying']]
['braincell', ['feel']]
['sure', ['not']]
['tho', []]
Review Number :  786
[['virus', []], ['worse', ['made']], ['monegligence', []]]
['virus', []]

saikang
week
kena
hahahhahaha
dead
question
ask
able
give
accurate
answer
c9
f.
quite
low
much
servicesupport
n9
n-code
navy
personnel
eligible
ship
duties
basically
areas
educationlevel
result
test
ippt
sit
fitness
platoonippt
ipptgold
scs
ocs
worthy
type
real
leadership
gold
bayo
interview
gpa
platoon
idea
jc
students
thing
applies
peer
appraisal
qualities
impt
initiative
point
section
annoying
fieldcamp
normal
days
commanders
potential
people
imo
way
times
cmd
school
personally
command
saf
proven
right
unit
men
sgt
officer
forgetic
icya
bring
book
necessary
stuff
liao
light
mono
tekong
pack
undergarments
hand
washable
detergent
clothes
hangers
hygiene
products
shaver
creametc
powerbanks
snacks
probability
bunk
check
powder
black
tape
extra
padlock
legend
helpful
honestly
private
cos
uk
shitu
pass
q
fast
tho
covid
thru
find
instructor
military
driver
lucky
lessons
clear
less
money
earn
months
everything
gon
dirty
anyways
wash
muddy
bag
wet
bottle
etc
nasty
assaultpack
fieldpack
fuss


sight
sarg
peeps
cock
mc
downpes
commander
uncommon
cny
press
egoistic
kias
rest
plz
levelanalysis
ice
intensely
hurting
attention
aim
statusdowntime
heals
ultimately
pt
major
highkey
iirc
bookalr
serves
cluster
generally
arrange
dates
normally
center
s1
blooddraw
termstatus
status
ooc'ed
commandercourse
ligament
physio
oocguys
graduates
posted
guards
intakebatch
slot
sitguy
batches
pasirlaba
easily
bearable
jealous
receptionists
centres
deal
forward
pay
tmind
position
rn
fracture
unable
minimum
feel
painbro
connections
line
delay
cpc
deciding
prolly
suggest
avoid
activity
exacerbate
plspls
jobscope
difference
knee
advise
f
renders
incapable
exercises
alway
drag
re-bmt
relax
guessing
national
chiefdefense
defenseforce
decade
gear
gotten
aches/problems
singaporean
active
nimble
bodytime
skills
ie
logistic
civilian
bargain
gain
trust
material
useful
outside
treat
learningexperience
stupid
tear
pes-able
mechanisms
cpl
shittier
workingrelationship
perm
adhoc
hocnature
abit
unbearable
toll


In [10]:
#pd.set_option('display.max_rows', None)
final_sent2

,Avg_sent,Freq
’,1.000000,87
u,1.038095,84
time,1.047619,65
n't,1.000000,61
also,1.016949,53
ns,1.263158,43
pes,0.978261,41
people,0.956522,38
medical,1.050000,37
really,1.000000,37
